In [2]:
#intalling dependencies
import pandas as pd

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster


In [3]:
##cleaning data from SP GOV DATA
url ='https://www.saude.sp.gov.br/resources/cve-centro-de-vigilancia-epidemiologica/areas-de-vigilancia/doencas-de-transmissao-por-vetores-e-zoonoses/dados/dengue/2023/dengue23_se.htm'
listOfCases = pd.read_html(url)
df = listOfCases[0]
tt = df[4:]
cidade= tt.iloc[:, 6]
cidade.drop(cidade.index[-5:], inplace=True)
total = tt.iloc[:, -1]
total.drop(total.index[-5:], inplace=True)
data = pd.DataFrame()
data['Cidade'] = cidade
data['Cidade'] = data['Cidade'].str.split(' ', n=1).str[1]
data['Total'] = total


In [4]:
#add geolocation on cities using GeoPy
locator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
data['Coords'] = data['Cidade'].apply(geocode)


In [5]:
#add latitude and longitude to values
data['lat'] = data['Coords'].apply(lambda loc: loc.latitude if loc else None)
data['lon'] = data['Coords'].apply(lambda loc: loc.longitude if loc else None)

In [6]:
#refactoring data
registerWithNaN = data.loc[data['lat'].isna()].sum() #moji das cruzes = total case -> 13 (wrong name of city)
location = locator.geocode('Mogi das Cruzes')

data.loc[data['Cidade'] == 'Moji das Cruzes', 'Coords'] = location[0]
data.loc[data['Cidade'] == 'Moji das Cruzes', 'lat'] = location.latitude
data.loc[data['Cidade'] == 'Moji das Cruzes', 'lon'] = location.longitude
data.loc[data['Cidade'] == 'Moji das Cruzes', 'Cidade'] = 'Mogi das Cruzes'



In [7]:
#removing outliers based on lat and lon in state of SP
locoutliers = data.copy()
locRemove = locoutliers.loc[((locoutliers['lat'] <= -19) & (locoutliers['lat'] >= -25)) & ((locoutliers['lon'] <= -44) & (locoutliers['lon'] >= -53))]

In [8]:
#duplicating values by total of cases in cities
qtdcoords = pd.DataFrame()

for i in range(len(locRemove)):
    qtdcoords['latG'] = locRemove['lat'].repeat(locRemove['Total'])

for i in range(len(locoutliers)):
    qtdcoords['lonG'] = locRemove['lon'].repeat(locRemove['Total'])


In [9]:
#Creating heat map using folium
heatMap = folium.Map(location=[-21.629235131241312, -49.09230350339458], zoom_start=6.5)  
listValues =  qtdcoords.values.tolist()
HeatMap(listValues, radius=10).add_to(heatMap)
heatMap.save('heatMap.html')
heatMap
print('oi')

oi


In [10]:
#Clustering FoliumMap using Cities and Cases
clusteringMap = folium.Map(location=[-21.629235131241312, -49.09230350339458], zoom_start=6.5)  
markerClustering = MarkerCluster().add_to(clusteringMap)
for i in range(0, len(locRemove)):
    popup = folium.Popup(locRemove.iloc[i]['Cidade'] + ' ' + locRemove.iloc[i]['Total'] + ' Casos', min_width=100, max_width=200)

    folium.Marker(location=[locRemove.iloc[i]['lat'], locRemove.iloc[i]['lon']], popup= popup).add_to(markerClustering)

clusteringMap.save('clusteringMap.html')
clusteringMap

